In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.metrics import f1_score, make_scorer
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('ilpd.csv')

# O objetivo dessa base é avaliar se uma pessoa vai ser um paciente hepático ou não. A feature que determina a classe nessa base é a “V7”?

In [ ]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


# Sobre as informações de número de features e amostras, marque a alternativa CORRETA. 




In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      583 non-null    int64  
 1   V2      583 non-null    object 
 2   V3      583 non-null    float64
 3   V4      583 non-null    float64
 4   V5      583 non-null    int64  
 5   V6      583 non-null    int64  
 6   V7      583 non-null    int64  
 7   V8      583 non-null    float64
 8   V9      583 non-null    float64
 9   V10     583 non-null    float64
 10  Class   583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


# Essa base conta com 3 atributos categóricos? Precisamos fazer o pré processamento desses atributos para que todos sejam numéricos?

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      583 non-null    int64  
 1   V2      583 non-null    object 
 2   V3      583 non-null    float64
 3   V4      583 non-null    float64
 4   V5      583 non-null    int64  
 5   V6      583 non-null    int64  
 6   V7      583 non-null    int64  
 7   V8      583 non-null    float64
 8   V9      583 non-null    float64
 9   V10     583 non-null    float64
 10  Class   583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


# A base não tem dados faltantes?

In [ ]:
df.describe()

,V1,V3,V4,V5,V6,V7,V8,V9,V10,Class
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000
mean,44.746141,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.947064,1.286449
std,16.189833,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.318492,0.452490
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000,1.000000
25%,33.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000,1.000000
50%,45.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.947064,1.000000
75%,58.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000,2.000000
max,90.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000,2.000000


In [ ]:
df.isna().sum()

V1       0
V2       0
V3       0
V4       0
V5       0
V6       0
V7       0
V8       0
V9       0
V10      0
Class    0
dtype: int64

# Sobre o best_score_ encontrado durante a otimização do SVC, marque a alternativa CORRETA. 

In [ ]:
# quantidade percentual de amostras para 'Male' e 'Female' na coluna V2
df['V2'].value_counts(normalize=True)

Male      0.756432
Female    0.243568
Name: V2, dtype: float64

In [ ]:
# criando um dicionario de dados para o mapeamento
name_to_class = {
    'Male': 1,
    'Female': 0
}

# substituindo os valores categóricos pelo mapeamento
df['V2'] = df['V2'].map(name_to_class)

#check
df.head(5)



,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [ ]:
class_mapping = {
    1:1,
    2:0
}

# substituindo os valores categóricos pelo mapeamento
df['Class'] = df['Class'].map(class_mapping)

#check
df.head(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [ ]:
#transforma os dados em array
data = df.iloc[:, :-1].values  #dados de entrada
labels = df.iloc[:,10].values  # saídas ou target

In [ ]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions_svc = dict(kernel = ['sigmoid', 'poly', 'rbf'], 
                     C = uniform(loc=1, scale=10))

#instânciando meu classificador
classifier = SVC()

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
random_cv_svc = RandomizedSearchCV(classifier, distributions_svc, cv = cv_strat, scoring = f1, random_state = 246, n_iter = 5)
random_cv_svc.fit(data, labels)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=5, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe931898bd0>,
                                        'kernel': ['sigmoid', 'poly', 'rbf']},
                   pre_dispatch='2*n_jobs', random_state=246, refit=True,
                   return_train_score=False, scoring=make_scorer(f1_score),
                   verbose=0)

In [ ]:
print('Melhor resultado f1:', random_cv_svc.best_score_)

Melhor resultado f1: 0.832818081808181


# O best_params_ encontrado durante a otimização do SVC indica que a melhor combinação de parâmetros usa um kernel RBF. 

In [ ]:
print('\n\nMelhor configuração de kernel:', random_cv_svc.best_params_['kernel'])



Melhor configuração de kernel: rbf


# O best_params_ encontrado durante a otimização do SVC indica que C encontrado é aproximadamente 5. 

In [ ]:
print('\n\nMelhor configuração de C:', random_cv_svc.best_params_['C'])



Melhor configuração de C: 6.893808746119847


# A melhor combinação de best_params_ encontrado durante a otimização do SVC é: 

In [ ]:
print('\n\nMelhor configuração de hiperparâmetros:', random_cv_svc.best_params_)



Melhor configuração de hiperparâmetros: {'C': 6.893808746119847, 'kernel': 'rbf'}


In [ ]:
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo Random Search: \n', random_cv_svc.best_estimator_)



Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo Random Search: 
 SVC(C=6.893808746119847, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


# Sobre o best_score_ encontrado durante a otimização do Random Forest, marque a alternativa CORRETA. 

In [ ]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions_rf = dict(n_estimators = randint(10, 1000),
                      bootstrap = [True, False],
                      criterion = ['entropy','gini'])

#instânciando meu classificador
classifier_rf = RandomForestClassifier(random_state = 54)

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
random_rf = RandomizedSearchCV(classifier_rf, distributions_rf, cv = cv_strat, scoring = f1, random_state = 54, n_iter = 5)
random_rf.fit(data, labels)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_s...
                     

In [ ]:
print('Melhor resultado f1:', random_rf.best_score_)

Melhor resultado f1: 0.7971841519245542


# O best_params_ encontrado durante a otimização do Random Forest indica que a melhor combinação de parâmetros usa Boostrap True. 

In [ ]:
print('\n\nMelhor configuração de bootstrap:', random_rf.best_params_['bootstrap'])



Melhor configuração de bootstrap: True


# O best_params_ encontrado durante a otimização do Random Forest indica que a melhor combinação de parâmetros usa Criterion Entropy. 

In [ ]:
print('\n\nMelhor configuração de criterion:', random_rf.best_params_['criterion'])



Melhor configuração de criterion: entropy


# O best_params_ encontrado durante a otimização do Random Forest indica que a melhor combinação de parâmetros conta com quantos estimadores? 

In [ ]:
print('\n\nMelhor configuração de n_estimators:', random_rf.best_params_)



Melhor configuração de n_estimators: {'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 913}
